In [1]:
import numpy as np
import torch
from models import *
from fid import get_fid

from torch.utils.data import TensorDataset, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/f

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use keras.layers.flatten instead.


In [2]:
G_models = ['cifar10_dcgan/G_25.pth', 'cifar10_bigan/G_25.pth', 'cifar10_logan_b/G_25.pth']
E_models = ['cifar10_dcgan/Epost_1.pth', 'cifar10_bigan/Epost_1.pth', 'cifar10_logan_b/Epost_1.pth']

In [3]:
num_images = 1000

In [4]:
dataset = CIFAR10(root='.', download=True, transform=Compose([Resize(32), ToTensor(), Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]))
dataloader = DataLoader(dataset, batch_size=num_images, shuffle=True)

Files already downloaded and verified


In [5]:
for (G_file, E_file) in zip(G_models, E_models):
    G = torch.load(G_file).to('cuda:1').eval()
    E = torch.load(E_file).to('cuda:1').eval()
    
    images1 = np.zeros((num_images, 3, 32, 32))
    images2 = np.zeros((num_images, 3, 32, 32))
    
    real = ((next(iter(dataloader))[0]+1)*127).squeeze().numpy()
    images1[:,:,:,:] = real.copy()
    images1 = images1.astype(np.uint8)
    
    real = next(iter(dataloader))[0]
    latents = E(real.to('cuda:1')).detach()
    recon = ((G(latents).cpu().detach()+1)*127).squeeze().numpy()
    images2[:,:,:,:] = recon.copy()
    images2 = images2.astype(np.uint8)
    
    print(G_file, get_fid(images1, images2))

Calculating FID with 1000 images from each distribution
FID calculation time: 132.610401 s
cifar10_dcgan/G_25.pth 227.70172
Calculating FID with 1000 images from each distribution
FID calculation time: 133.400751 s
cifar10_bigan/G_25.pth 426.721
Calculating FID with 1000 images from each distribution
FID calculation time: 98.156068 s
cifar10_logan_b/G_25.pth 140.63152
